In [1]:
# !~/tf-nvidia/bin/pip3 install git+https://github.com/huseinzol05/tensor2tensor.git --no-deps

In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [2]:
from tensor2tensor import models
from tensor2tensor import problems
from tensor2tensor.layers import common_layers
from tensor2tensor.utils import trainer_lib
from tensor2tensor.utils import t2t_model
from tensor2tensor.utils import registry
from tensor2tensor.utils import metrics
from tensor2tensor.data_generators import problem
from tensor2tensor.data_generators import text_problems
from tensor2tensor.data_generators import translate
from tensor2tensor.utils import registry

2022-07-31 16:49:52.774700: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0



/home/ubuntu/tf-nvidia/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import youtokentome as yttm

In [4]:
bpe = yttm.BPE(model='rumi-jawi.yttm')

In [5]:
class Encoder:
    def __init__(self, bpe):
        self.bpe = bpe
        self.vocab_size = len(self.bpe.vocab())

    def encode(self, s):
        s = self.bpe.encode(s, output_type=yttm.OutputType.ID)
        s = [i + [1] for i in s]
        return s

    def decode(self, ids, strip_extraneous=False):
        return self.bpe.decode(list(ids))[0]

In [6]:
@registry.register_problem
class Jawi(text_problems.Text2TextProblem):
    @property
    def approx_vocab_size(self):
        return 32000

    @property
    def is_generate_per_split(self):
        # generate_data will shard the data into TRAIN and EVAL for us.
        return False

    @property
    def dataset_splits(self):
        return [
            {'split': problem.DatasetSplit.TRAIN, 'shards': 200},
            {'split': problem.DatasetSplit.EVAL, 'shards': 1},
        ]

    def feature_encoders(self, data_dir):
        s_encoder = Encoder(bpe)
        return {'inputs': s_encoder, 'targets': s_encoder}

In [7]:
PROBLEM = 'jawi'
problem = problems.problem(PROBLEM)

In [8]:
class Model:
    def __init__(self, HPARAMS = "transformer_base", DATA_DIR = 't2t/data2'):
        
        self.X = tf.placeholder(tf.int32, [None, None])
        self.Y = tf.placeholder(tf.int32, [None, None])
        self.beam_size = tf.placeholder(tf.int32, shape=(), name = 'beam_size')
        self.temperature = tf.placeholder(tf.float32, shape=(), name = 'temperature')
        
        self.X_seq_len = tf.count_nonzero(self.X, 1, dtype=tf.int32)
        maxlen_decode = 50 + tf.reduce_max(self.X_seq_len)
        
        x = tf.expand_dims(tf.expand_dims(self.X, -1), -1)
        y = tf.expand_dims(tf.expand_dims(self.Y, -1), -1)
        
        features = {
            "inputs": x,
            "targets": y,
            "target_space_id": tf.constant(1, dtype=tf.int32),
        }
        print(features)
        
        Modes = tf.estimator.ModeKeys
        hparams = trainer_lib.create_hparams(HPARAMS, data_dir=DATA_DIR, problem_name=PROBLEM)
        translate_model = registry.model('transformer')(hparams, Modes.PREDICT)
        logits, _ = translate_model(features)
        
        with tf.variable_scope(tf.get_variable_scope(), reuse=True):
            self.fast_result = translate_model._greedy_infer(features, maxlen_decode)["outputs"]
            self.beam_result = translate_model._beam_decode_slow(
                features, maxlen_decode, beam_size=self.beam_size, 
                top_beams=1, alpha=self.temperature)["outputs"]
        
        self.fast_result = tf.identity(self.fast_result, name = 'greedy')
        self.beam_result = tf.identity(self.beam_result, name = 'beam')

In [9]:
import tensorflow as tf

In [10]:
ckpt_path = tf.train.latest_checkpoint('t2t-jawi-rumi/base')
ckpt_path

't2t-jawi-rumi/base/model.ckpt-200000'

In [11]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model()


2022-07-31 16:50:00.350836: I tensorflow/core/platform/profile_utils/cpu_utils.cc:109] CPU Frequency: 2496000000 Hz
2022-07-31 16:50:00.351615: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x67cf890 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-07-31 16:50:00.351628: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2022-07-31 16:50:00.353616: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-07-31 16:50:00.478191: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1082] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-31 16:50:00.479007: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x42d5120 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2022-07-31 16:50

2022-07-31 16:50:01.431786: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1206] Device interconnect StreamExecutor with strength 1 edge matrix:
2022-07-31 16:50:01.431812: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1212]      0 
2022-07-31 16:50:01.431817: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1225] 0:   N 
2022-07-31 16:50:01.432482: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1082] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-31 16:50:01.433391: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1082] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-31 16:50:01.434206: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1351] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 22064 MB memory) -> physical GPU (device: 0, name: NVIDIA GeF

{'inputs': <tf.Tensor 'ExpandDims_1:0' shape=(?, ?, 1, 1) dtype=int32>, 'targets': <tf.Tensor 'ExpandDims_3:0' shape=(?, ?, 1, 1) dtype=int32>, 'target_space_id': <tf.Tensor 'Const_1:0' shape=() dtype=int32>}
INFO:tensorflow:Setting T2TModel mode to 'infer'


INFO:tensorflow:Setting T2TModel mode to 'infer'


INFO:tensorflow:Setting hparams.dropout to 0.0


...

INFO:tensorflow:Transforming feature 'targets' with symbol_modality_32000_512.targets_bottom


INFO:tensorflow:Building model body


INFO:tensorflow:Building model body


INFO:tensorflow:Transforming body output with symbol_modality_32000_512.top


INFO:tensorflow:Transforming body output with symbol_modality_32000_512.top


In [12]:
var_lists = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)
saver = tf.train.Saver(var_list = var_lists)
saver.restore(sess, ckpt_path)

INFO:tensorflow:Restoring parameters from t2t-jawi-rumi/base/model.ckpt-200000


INFO:tensorflow:Restoring parameters from t2t-jawi-rumi/base/model.ckpt-200000


In [13]:
import json

train_X, train_Y = [], []
with open('../jawi-rumi/jawi-rumi-news-full.test') as fopen:
    for line in fopen:
        d = json.loads(line)
        train_X.append(d[1])
        train_Y.append(d[0])

In [14]:
train_X[:5]

['لاڤورن سبواه ڤورتل بريتا بارو-بارو اين ممتيق سأورڠ اهلي ڤرليمين ڤقر يڠ اڠڬن ديناماكن يڠ مندعوا عزمين',
 'ايسو بيل تنب دباوا ك كابينيت - صيفالدين',
 'دڠن ممبوات سمڤل مراسا كينياڠ لبيه چڤت.',
 'باڽق اورڠ يڠ مراسا تاكوت بهوا اي اكن مڠڬيسير ڤران مأنسي.',
 'سيڠاڤورا، منوروت كمنترين ترنسڤورتاسي سيڠاڤورا، تيدق اكن راڬو اونتوق مڠمبيل تيندقن تڬس اتس اكتيۏيتس ترسبوت.']

In [15]:
encoder = Encoder(bpe)

In [16]:
encoder.encode([train_X[0]])

[[3482,
  1220,
  9951,
  5267,
  8175,
  264,
  25218,
  1824,
  1101,
  1561,
  6975,
  216,
  12117,
  9523,
  216,
  6701,
  13289,
  1]]

In [17]:
string = train_X[0]
encoded = encoder.encode([string])
f, b = sess.run([model.fast_result, model.beam_result], feed_dict = {model.X: encoded, model.temperature: 0.5,
                                                                    model.beam_size: 2})
encoder.decode(f[0].tolist()), encoder.decode(b[0].tolist())

2022-07-31 16:50:21.084705: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11


('laporan sebuah portal berita baru-baru ini memetik seorang ahli parlimen pkr yang enggan dinamakan yang mendakwa azmin<UNK>',
 'laporan sebuah portal berita baru-baru ini memetik seorang ahli parlimen pkr yang enggan dinamakan yang mendakwa azmin<UNK>')

In [18]:
string = train_X[1]
encoded = encoder.encode([string])
f, b = sess.run([model.fast_result, model.beam_result], feed_dict = {model.X: encoded, model.temperature: 0.5,
                                                                    model.beam_size: 2})
encoder.decode(f[0].tolist()), encoder.decode(b[0].tolist())

('isu bil tnb dibawa ke kabinet - saifuddin<UNK>',
 'isu bil tnb dibawa ke kabinet - saifuddin<UNK>')

In [19]:
train_X[1]

'ايسو بيل تنب دباوا ك كابينيت - صيفالدين'

In [20]:
string = train_X[2]
encoded = encoder.encode([string])
f, b = sess.run([model.fast_result, model.beam_result], feed_dict = {model.X: encoded, model.temperature: 0.5,
                                                                    model.beam_size: 2})
encoder.decode(f[0].tolist()), encoder.decode(b[0].tolist())

('dengan membuat sampel merasa kenyang lebih cepat.<UNK>',
 'dengan membuat sampel merasa kenyang lebih cepat.<UNK>')

In [21]:
f[0][f[0] > 3].tolist()

[359, 2176, 20082, 17463, 1529, 5258, 690, 8414, 46]

In [22]:
encoder.decode([0,1,2,3])

'<PAD><UNK><BOS><EOS>'

In [23]:
def calculate_cer(actual, hyp):
    """
    Calculate CER using `python-Levenshtein`.
    """
    import Levenshtein as Lev

    actual = actual.replace(' ', '')
    hyp = hyp.replace(' ', '')
    return Lev.distance(actual, hyp) / len(actual)


def calculate_wer(actual, hyp):
    """
    Calculate WER using `python-Levenshtein`.
    """
    import Levenshtein as Lev

    b = set(actual.split() + hyp.split())
    word2char = dict(zip(b, range(len(b))))

    w1 = [chr(word2char[w]) for w in actual.split()]
    w2 = [chr(word2char[w]) for w in hyp.split()]

    return Lev.distance(''.join(w1), ''.join(w2)) / len(actual.split())

In [24]:
string = train_X[1]
encoded = encoder.encode([string])
f = sess.run(model.fast_result, feed_dict = {model.X: encoded})[0]
p = encoder.decode(f[f > 3].tolist())

In [25]:
calculate_cer(train_Y[1], p), calculate_wer(train_Y[1], p)

(0.0, 0.0)

In [29]:
from tqdm import tqdm

cers, wers = [], []
for i in tqdm(range(len(train_X[:10000]))):
    string = train_X[i]
    encoded = encoder.encode([string])
    f = sess.run(model.fast_result, feed_dict = {model.X: encoded})[0]
    p = encoder.decode(f[f > 3].tolist())
    cer, wer = calculate_cer(train_Y[i], p), calculate_wer(train_Y[i], p)
    cers.append(cer)
    wers.append(wer)

100%|████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [2:00:36<00:00,  1.38it/s]


In [30]:
import numpy as np

np.mean(cers), np.mean(wers)

(0.0007639846016540444, 0.003042474270655385)

In [34]:
!rm -rf jawi-rumi-en-ms-base

In [35]:
saver = tf.train.Saver(tf.trainable_variables())
saver.save(sess, 'jawi-rumi-base/model.ckpt')

'jawi-rumi-base/model.ckpt'

In [36]:
strings = ','.join(
    [
        n.name
        for n in tf.get_default_graph().as_graph_def().node
        if ('Variable' in n.op
        or 'Placeholder' in n.name
        or 'greedy' in n.name
        or 'beam' in n.name
        or 'alphas' in n.name
        or 'temperature' in n.name
        or 'beam_size' in n.name
        or 'self/Softmax' in n.name)
        and 'adam' not in n.name
        and 'beta' not in n.name
        and 'global_step' not in n.name
        and 'modality' not in n.name
        and 'Assign' not in n.name
    ]
)
strings.split(',')

['Placeholder',
 'Placeholder_1',
 'beam_size',
 'temperature',
 'transformer/body/target_space_embedding/kernel/Read/ReadVariableOp',
 'transformer/parallel_0_4/transformer/transformer/body/target_space_embedding/ReadVariableOp',
 'transformer/body/encoder/layer_0/self_attention/layer_prepostprocess/layer_norm/layer_norm_scale/Read/ReadVariableOp',
 'transformer/body/encoder/layer_0/self_attention/layer_prepostprocess/layer_norm/layer_norm_bias/Read/ReadVariableOp',
 'transformer/parallel_0_4/transformer/transformer/body/encoder/layer_0/self_attention/layer_prepostprocess/layer_norm/ReadVariableOp',
 'transformer/parallel_0_4/transformer/transformer/body/encoder/layer_0/self_attention/layer_prepostprocess/layer_norm/ReadVariableOp_1',
 'transformer/body/encoder/layer_0/self_attention/multihead_attention/q/kernel/Read/ReadVariableOp',
 'transformer/parallel_0_4/transformer/transformer/body/encoder/layer_0/self_attention/multihead_attention/q/Tensordot/ReadVariableOp',
 'transformer/bod

In [37]:
def freeze_graph(model_dir, output_node_names):

    if not tf.gfile.Exists(model_dir):
        raise AssertionError(
            "Export directory doesn't exists. Please specify an export "
            'directory: %s' % model_dir
        )

    checkpoint = tf.train.get_checkpoint_state(model_dir)
    input_checkpoint = checkpoint.model_checkpoint_path

    absolute_model_dir = '/'.join(input_checkpoint.split('/')[:-1])
    output_graph = absolute_model_dir + '/frozen_model.pb'
    clear_devices = True
    with tf.Session(graph = tf.Graph()) as sess:
        saver = tf.train.import_meta_graph(
            input_checkpoint + '.meta', clear_devices = clear_devices
        )
        saver.restore(sess, input_checkpoint)
        output_graph_def = tf.graph_util.convert_variables_to_constants(
            sess,
            tf.get_default_graph().as_graph_def(),
            output_node_names.split(','),
        )
        with tf.gfile.GFile(output_graph, 'wb') as f:
            f.write(output_graph_def.SerializeToString())
        print('%d ops in the final graph.' % len(output_graph_def.node))

In [38]:
freeze_graph('jawi-rumi-base', strings)

2022-07-31 20:35:36.252254: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1082] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-31 20:35:36.252508: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1666] Found device 0 with properties: 
name: NVIDIA GeForce RTX 3090 Ti major: 8 minor: 6 memoryClockRate(GHz): 1.86
pciBusID: 0000:01:00.0
2022-07-31 20:35:36.252550: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-07-31 20:35:36.252592: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2022-07-31 20:35:36.252612: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcufft.so.10
2022-07-31 20:35:36.252626: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library

INFO:tensorflow:Restoring parameters from jawi-rumi-base/model.ckpt


INFO:tensorflow:Restoring parameters from jawi-rumi-base/model.ckpt


INFO:tensorflow:Froze 201 variables.


INFO:tensorflow:Froze 201 variables.


INFO:tensorflow:Converted 201 variables to const ops.


INFO:tensorflow:Converted 201 variables to const ops.


11017 ops in the final graph.


In [39]:
def load_graph(frozen_graph_filename):
    with tf.gfile.GFile(frozen_graph_filename, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def)
    return graph

In [40]:
g = load_graph('jawi-rumi-base/frozen_model.pb')
x = g.get_tensor_by_name('import/Placeholder:0')
greedy = g.get_tensor_by_name('import/greedy:0')
beam = g.get_tensor_by_name('import/beam:0')
test_sess = tf.InteractiveSession(graph = g)


2022-07-31 20:36:02.737820: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1082] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-31 20:36:02.737991: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1666] Found device 0 with properties: 
name: NVIDIA GeForce RTX 3090 Ti major: 8 minor: 6 memoryClockRate(GHz): 1.86
pciBusID: 0000:01:00.0
2022-07-31 20:36:02.738014: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-07-31 20:36:02.738035: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2022-07-31 20:36:02.738042: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcufft.so.10
2022-07-31 20:36:02.738048: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic librar

In [41]:
from tensorflow.tools.graph_transforms import TransformGraph

In [42]:
transforms = ['add_default_attributes',
             'remove_nodes(op=Identity, op=CheckNumerics, op=Dropout)',
             'fold_constants(ignore_errors=true)',
             'fold_batch_norms',
             'fold_old_batch_norms',
             'quantize_weights(fallback_min=-10, fallback_max=10)',
             'strip_unused_nodes',
             'sort_by_execution_order']

pb = 'jawi-rumi-base/frozen_model.pb'
input_graph_def = tf.GraphDef()
with tf.gfile.FastGFile(pb, 'rb') as f:
    input_graph_def.ParseFromString(f.read())

print(pb)

transformed_graph_def = TransformGraph(input_graph_def, 
                                       ['Placeholder'],
                                       ['greedy', 'beam'], transforms)

with tf.gfile.GFile(f'{pb}.quantized', 'wb') as f:
    f.write(transformed_graph_def.SerializeToString())

jawi-rumi-base/frozen_model.pb


2022-07-31 20:36:10.991916: I tensorflow/tools/graph_transforms/transform_graph.cc:318] Applying add_default_attributes
2022-07-31 20:36:11.089777: I tensorflow/tools/graph_transforms/transform_graph.cc:318] Applying remove_nodes
2022-07-31 20:36:11.166402: I tensorflow/tools/graph_transforms/remove_nodes.cc:78] Skipping replacement for greedy
2022-07-31 20:36:11.166446: I tensorflow/tools/graph_transforms/remove_nodes.cc:78] Skipping replacement for beam
2022-07-31 20:36:11.276492: I tensorflow/tools/graph_transforms/remove_nodes.cc:78] Skipping replacement for greedy
2022-07-31 20:36:11.276540: I tensorflow/tools/graph_transforms/remove_nodes.cc:78] Skipping replacement for beam
2022-07-31 20:36:11.379294: I tensorflow/tools/graph_transforms/remove_nodes.cc:78] Skipping replacement for greedy
2022-07-31 20:36:11.379351: I tensorflow/tools/graph_transforms/remove_nodes.cc:78] Skipping replacement for beam
2022-07-31 20:36:11.471055: I tensorflow/tools/graph_transforms/remove_nodes.cc:

In [43]:
from malaya_boilerplate.huggingface import upload_dict

In [44]:
!cp t2t-jawi-rumi/base/*.tfevents.* jawi-rumi-base
!tar -cvf jawi-rumi-base.tar jawi-rumi-base

jawi-rumi-base/
jawi-rumi-base/checkpoint
jawi-rumi-base/frozen_model.pb.quantized
jawi-rumi-base/model.ckpt.index
jawi-rumi-base/model.ckpt.data-00000-of-00001
jawi-rumi-base/events.out.tfevents.1659204642.huseincomel-desktop
jawi-rumi-base/model.ckpt.meta
jawi-rumi-base/frozen_model.pb


In [45]:
files_mapping = {'jawi-rumi-base.tar': 'jawi-rumi-base.tar'}
upload_dict(model = 'pretrained-jawi-rumi', files_mapping = files_mapping)

/home/ubuntu/tf-nvidia/lib/python3.8/site-packages/huggingface_hub/hf_api.py:79: FutureWarning: `name` and `organization` input arguments are deprecated and will be removed in v0.8. Pass `repo_id` instead.
  warnings.warn(


In [46]:
files_mapping = {'jawi-rumi-base/frozen_model.pb': 'model.pb'}
upload_dict(model = 'jawi-rumi-base', files_mapping = files_mapping)

In [47]:
files_mapping = {'jawi-rumi-base/frozen_model.pb.quantized': 'model.pb'}
upload_dict(model = 'jawi-rumi-base-quantized', files_mapping = files_mapping)